In [34]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait

import time
import pandas as pd

In [43]:
def get_jobs(num_jobs, verbose, path):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    url = "https://www.glassdoor.com/Job/data-analyst-jobs-SRCH_KO0,12.htm?minSalary=15000&includeNoSalaryJobs=false&maxSalary=210200"
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(15)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
            print('x worked')
        except NoSuchElementException:
            print('x failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [46]:

df1= get_jobs(400, False,r'C:/Users/kaout/Downloads/salary_project/chromedriver')
df1

x worked
Progress: 0/400
Progress: 1/400
Progress: 2/400
Progress: 3/400
Progress: 4/400
Progress: 5/400
Progress: 6/400
Progress: 7/400
Progress: 8/400
Progress: 9/400
Progress: 10/400
Progress: 11/400
Progress: 12/400
Progress: 13/400
Progress: 14/400
Progress: 15/400
Progress: 16/400
Progress: 17/400
Progress: 18/400
Progress: 19/400
Progress: 20/400
Progress: 21/400
Progress: 22/400
Progress: 23/400
Progress: 24/400
Progress: 25/400
Progress: 26/400
Progress: 27/400
Progress: 28/400
Progress: 29/400
x failed
Progress: 30/400
Progress: 31/400
Progress: 32/400
Progress: 33/400
Progress: 34/400
Progress: 35/400
Progress: 36/400
Progress: 37/400
Progress: 38/400
Progress: 39/400
Progress: 40/400
Progress: 41/400
Progress: 42/400
Progress: 43/400
Progress: 44/400
Progress: 45/400
Progress: 46/400
Progress: 47/400
Progress: 48/400
Progress: 49/400
Progress: 50/400
Progress: 51/400
Progress: 52/400
Progress: 53/400
Progress: 54/400
Progress: 55/400
Progress: 56/400
Progress: 57/400
Progre

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Marketing Data Analyst,$48K-$83K (Glassdoor est.),Description/Responsibilities\n\n\nPosition Sum...,3.1,Tivity Health\n3.1,"Fort Washington, PA",-1,501 to 1000 Employees,1981,Company - Public,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1
1,Marketing Data Analyst,$48K-$83K (Glassdoor est.),Can you work in a relationship with other peop...,4.2,Idaho Central Credit Union\n4.2,"Chubbuck, ID",-1,1001 to 5000 Employees,1940,Nonprofit Organization,Banks & Credit Unions,Finance,$50 to $100 million (USD),-1
2,Data Analyst,$48K-$83K (Glassdoor est.),"Rational is growing, and we need amazing talen...",2.7,Rational\n2.7,"Redmond, WA",-1,201 to 500 Employees,2009,Company - Public,Consulting,Business Services,Unknown / Non-Applicable,-1
3,Tactical Data Link (TDL) Analyst,$48K-$83K (Glassdoor est.),Overview:\n\nOasis Systems has an exciting opp...,3.7,Oasis Systems LLC\n3.7,"Hampton, VA",-1,1001 to 5000 Employees,1997,Company - Private,IT Services,Information Technology,$100 to $500 million (USD),-1
4,Data Scientist / Analyst Virtual Hiring Event,$48K-$83K (Glassdoor est.),Scientific Research Corporation Hiring Event!\...,4.2,Scientific Research Corporation\n4.2,"Charleston, SC",-1,1001 to 5000 Employees,1988,Company - Private,IT Services,Information Technology,$100 to $500 million (USD),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Data Analyst,$43K-$123K (Glassdoor est.),Grant Thornton is seeking an Associate to join...,3.8,Grant Thornton\n3.8,"Arlington, VA",-1,5001 to 10000 Employees,1924,Company - Private,Accounting,Accounting & Legal,$1 to $2 billion (USD),-1
396,Data Analyst,$43K-$123K (Glassdoor est.),Overview\n\n\nNuvance Health is a family of aw...,2.6,Health Quest\n2.6,"Lagrangeville, NY",-1,1001 to 5000 Employees,-1,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$500 million to $1 billion (USD),-1
397,Data Analyst - Merch/Food and Bev,$43K-$123K (Glassdoor est.),Description:\n\nJOIN US AS A DATA ANALYST – ME...,3.5,TARGET\n3.5,"Minneapolis, MN",-1,10000+ Employees,1962,Company - Public,General Merchandise & Superstores,Retail,$10+ billion (USD),-1
398,Supply Chain Data Analyst,$43K-$123K (Glassdoor est.),"DO WORK THAT MATTERS\n\nAt Abbott, diverse\nid...",3.6,Abbott Laboratories\n3.6,"Waukegan, IL",-1,10000+ Employees,1888,Company - Public,Health Care Services & Hospitals,Health Care,$10+ billion (USD),-1


In [47]:
df1.to_csv('data_analyst_jobs.csv', index =False)
